# Tutorial 3: Insufficient Data (2D Range-only)

**API Version: Julia NavAbilitySDK.jl**

**Keywords:** Range-only, factor graph, under-determined, localization, mapping, SLAM, missing data, insufficient data, robotics, inference, Bayes tree, junction tree

## Overview

This example shows how non-Gaussian solutions occur from weak observability of desired variables from available data.  For this tutorial, we imagine a robot traveling around on a flat surface, in a rectangular trajectory making range measurements to beacons.  Let's assume WiFi power levels are used as a proxy for range measurements in this case.

<img src="https://github.com/NavAbility/BinderNotebooks/raw/main/static/icra-3/example-wifi.png" width=800>

To illustrate the the underdetermined solution aspect, we simplify the problem in that WiFi ranging is assumed to produce a pure unimodal (Gaussian) measurement.  This tutorial will show how non-Gaussian behavior can arise because tha variables are weakly or underconstrained by the available measurement dimensions.  This tutorial is designed such that this occurs throughout the entire problem.

We build a factor graph in stages as the robot moves around the environment through pose/keyframe epochs.  The factor graph will be solved with the [open core Caesar.jl solver](https://github.com/JuliaRobotics/Caesar.jl) at each pose epoch, which produces the posterior marginal beliefs on each of the variables in the system.  After each pose epoch solution, we will look at the marginal belief estimates of all the variables in the system.

We assume the robot is traveling on a XY plane, starting at the origin along X and turning left along Y, then negative X, negative Y back to the origin.  Only four WiFi beacons are in the environment where the robot is moving.  Measurements to the WiFi beacons can only be included with there if the signal is within range.  Two of the beacon locations are known as prior information, while the other two beacons are at an unknown location and their location will be eastimated simultaneously to the robot pose location in the same factor graph system -- making this a simultaneous localization and mapping (SLAM) problem.

To further simplify the tutorial, the "odometry" measurement between consecutive poses  are also taken as range-only (i.e. distance-only) measurements.  These "odometry" factors provide less information than conventional wheel or visual odometry constraints might provide.

This tutorial shows [one of four mechanisms](https://juliarobotics.org/Caesar.jl/latest/concepts/why_nongaussian/) that can intoduce non-Gaussian behavior into a factor graph system, see other examples for other mechanisms.  Note that the techniques used in this tutorial can readily be combined with with methods from other tutorials.  For example, the pure Gaussian measurement ranging models used here can be replaced with ambiguous measurements shown in ICRA Tutorial 2.  Or, can be combined with uncertain data association (i,e. multi-hypothesis) measurement models for unknown beacon associations similat to the technique used in ICRA Tutorial 4.  Learn more from our peer-reviewed publications listed here: `CJLDocs/Literature`

### Signatures Used

`Point2`, `PriorPoint2`, `Point2Point2Range`, `MvNormal`, `Normal`.

## Loading The Data

The ground truth positions for vehicle positions GTp and landmark positions GTl can be loaded into memory directly with these values.  **Note,** we are using variable names
- `l1, l2` as ranging beacons with known locations,
- `l3, l4` as ranging beacons with initially unknown locations,  

These beacon landmarks must be in range before measurements can be made to them.  For the tutorial, we imagine a robot moving from one position to the next in the XY space between the landmarks.  We use ground truth positions to build the simulation, while the SLAM solution has to resolve estimates of the variables as the main exercise of the tutorial.  The robot positions are denoted as
- `x0, x1, ...`.

Ground truth data is as follows:

In [ ]:
# Our dictionary of vehicle positions
GTp = Dict{Symbol, Vector{Float64}}()
GTp[:x0] = [0.0;0]
GTp[:x1] = [50.0;0]
GTp[:x2] = [100.0;0]
GTp[:x3] = [100.0;50.0]
GTp[:x4] = [100.0;100.0]
GTp[:x5] = [50.0;100.0]
GTp[:x6] = [0.0;100.0]
GTp[:x7] = [0.0;50.0]
GTp[:x8] = [0.0;-50.0]

# Our dictionary of landmark positions
GTl = Dict{Symbol, Vector{Float64}}()
GTl[:l1] = [10.0;30]
GTl[:l2] = [30.0;-30]
GTl[:l3] = [80.0;40]
GTl[:l4] = [120.0;-50];

## Loading the Necessary Packages

An optional install line is kept here in case the packages are not yet installed in your environment for whatever reason.

In [ ]:
# optional install of packages, in case they are not available in your environment for whatever reason
import Pkg; Pkg.add("NavAbilitySDK");

Load the necessary packages,

In [ ]:
using NavAbilitySDK

## Creating the Factor Graph

After loading the requried packages, lets start creating the factor graph using variables of type `Point2` (a.k.a. `Postion2`).

In [ ]:
# you need a unique userId:robotId, and can keep using that across all tutorials
userId = "Guest"
robotId = "SDKjl_"*(string(uuid4())[1:4])

# also create a client connection
client = NavAbilityHttpsClient()

In [ ]:
# You'll need a unique session number each time you run a new graph
sessionId = "Tutorial3_"*(string(uuid4())[1:4])
# context is the object to use below
context = Client(userId,robotId,sessionId)

# convenience Tuple
nvapl = client, context;

In [ ]:
# Collect all the async responses and wait at the end
resultIds = Task[]

# first pose with no prior info about the initial numerical estimate
push!(resultIds, 
  addVariable(nvapl..., Variable("x0", :Point2))
);

# add three landmarks
for lbl in ["l1";"l2";"l3"]
  push!(resultIds, 
    addVariable(nvapl..., Variable(lbl, :Point2))
  );
end

The initial graph also has prior location information about each of the known beacons/landmarks `l1` and `l2`.  Let's go ahead and add those as factors:

In [ ]:
# put prior on l1
f = Factor("l1f1", "PriorPoint2", ["l1"], 
  PriorPoint2Data(
    Z=FullNormal( GTl[:l1], diagm(ones(2)) )
  )
)
push!(resultIds, addFactor(nvapl..., f));

# put prior on l2
f = Factor("l2f1", "PriorPoint2", ["l2"], 
  PriorPoint2Data(
    Z=FullNormal( GTl[:l2], diagm(ones(2)) )
  )
)
push!(resultIds, addFactor(nvapl..., f));

# wait to make sure all the new additions are ready
waitForCompletion(client, resultIds; expectedStatuses=["Complete"])

The `PriorPoint2` is assumed to be a multivariate normal distribution of covariance `diagm(ones(2))`. Note the API `PriorPoint2(::SamplableBelief)` accepts any of the distribution objects that the Caesar.jl libraries support -- this is discussed further in subsection [Various `SamplableBelief` Distribution types](https://juliarobotics.org/Caesar.jl/latest/concepts/dataassociation/#Various-SamplableBelief-Distribution-Types).

## Adding Range Measurements Between Variables

Next we connect the three range measurements from the vehicle location `x0` to the three beacon landmarks `l1`, `l2`, and `l3`, respectively – and consider that the range measurements are completely relative between the vehicle and beacon position estimates:

In [ ]:
# first range measurement from x0 to l1
rhoZ1 = norm(GTl[:l1]-GTp[:x0])
f = Factor("x0l1f1", "Point2Point2Range", ["x0";"l1"], 
  Point2Point2RangeData(
    range=Normal(rhoZ1, 2)
  )
)
push!(resultIds, addFactor(nvapl..., f));

# second range measurement from x0 to l2
rhoZ2 = norm(GTl[:l2]-GTp[:x0])
f = Factor("x0l2f1", "Point2Point2Range", ["x0";"l2"], 
  Point2Point2RangeData(
    range=Normal(rhoZ2, 2)
  )
)
push!(resultIds, addFactor(nvapl..., f));

# third range measurement from x0 to l3
rhoZ3 = norm(GTl[:l3]-GTp[:x0])
f = Factor("x0l3f1", "Point2Point2Range", ["x0";"l3"], 
  Point2Point2RangeData(
    range=Normal(rhoZ3, 2)
  )
)
push!(resultIds, addFactor(nvapl..., f));

The ranging measurement standard deviation of 2.0 or 3.0 is taken, assuming a Gaussian measurement assumption. Again, any distribution could have been used. The factor graph should look as follows:

In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
GraphVizApp(context)

The factor graph figure above shows the structure between variables and factors.  Note the two priors on `l1` and `l2`, because we have prior information telling us where those beacons are.  The first pose `x0` is only connected via the range measurements, with no prior info about the starting location.  Also no prior info about the location of beacon `l3`.

## Inference and Visualizations

At this point we can call the solver and interpret the first results:

In [ ]:
push!(resultIds,
  solveSession(client, context)
);

# Give it a few seconds JIT compiling during first run.
println("running solve...")

Visualization tools for awarenss via the NavAbilty WebApp, to show the numerical values contained in the factor graph solution.

First look at the two landmark positions `l1`, `l2` at `(10.0,30)`, `(30.0,-30)` respectively.

In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
# plotBelief(fg, [:l1;:l2], levels=5, c=["cyan"; "black"])
MapVizApp(context)

Click on the generated "Geometric Map" graphic above to see a visualization of latest numerical results.

### First Location is Bi-Modal After Solve

Similarly, the belief estimate for the first vehicle position `x0` is bi-modal, due to the intersection of two range measurements:

In [ ]:
# pl = plotBelief(fg, :x0, levels=5, c=["red"])
MapVizApp(context)

Note, the first 'robot' positon in this localization and mapping problem is not associated with by a prior factor of any kind.  The initial position could have been anywhere, but the two range measurements to known landmarks limited the uncertainty as shown in the plot above.

### Non-Gaussian Estimate of the Unknown Beacon `l3`

In contrast to the known beacons `l1` and `l2` which have unimodal position estimates in the solution (owing to the prior information/assumptions on each), the belief over the position of unknown landmark `l3` is simultaneously resolved to a posterior estimate:

In [ ]:
# pl = plotBelief(fg, :l3, levels=10, c=["pink"])
MapVizApp(context)

Notice how `l3`'s location belief (i.e. surveying/mapping) forms a ring around the only available measurement to `l3` from `x0`.  A unimodal solution for `l3` **does not exist**.  In conventional linear modeling, we might say the system is [singular](https://en.wikipedia.org/wiki/Invertible_matrix).

## Gaining and Losing Modes (i.e. Hypotheses)

Next consider the vehicle moving a distance of 50 units–-and by design the direction of travel is not known–-to the next true position. The video above gives away the vehicle position with the cyan line, showing travel in the shape of a lower case 'e'. The following function handles (pseudo odometry) factors as range-only between positions and range-only measurement factors to beacons as the vehice travels.

In [ ]:
# This is a helper function that simulates how the robot moves and measures between ground truth positions. 
function vehicle_drives(
    CliCon::Tuple, 
    from_lbl::String, 
    to_lbl::String, 
    GTp::Dict, GTl::Dict; 
    measurelimit=150.0
  )
  #
  # client, context = CliCon
  resIds = Task[]
  # example with stateless local instance, various stateful distributed architectures are supported
  currvar = listVariables(CliCon...) |> fetch
  if !(to_lbl in currvar)
    println("Adding new variable $to_lbl")
    push!(resIds, addVariable(CliCon..., Variable(to_lbl, :Point2)));
    # an odometry distance factor
    @show rho = norm(GTp[Symbol(from_lbl)] - GTp[Symbol(to_lbl)])
    f = Factor("$(from_lbl)$(to_lbl)f1", "Point2Point2Range", [from_lbl;to_lbl], 
          Point2Point2RangeData(
            range=Normal(rho, 3.0)
        ))
    push!(resIds, addFactor(CliCon..., f));
  else
    @warn "Variable node $to_lbl already in the factor graph."
  end
  beacons = string.(keys(GTl))
  for ll in beacons
    rho = norm(GTl[Symbol(ll)] - GTp[Symbol(to_lbl)])
    # Add measurements to beacons/landmarks if within limit
    if rho < measurelimit
      if !(ll in currvar)
        println("Adding variable vertex $ll, not yet in fgl<:AbstractDFG.")
        push!(resIds, addVariable(CliCon..., Variable(ll, :Point2)));
      end
      f = Factor("$(to_lbl)$(ll)f1", "Point2Point2Range", [to_lbl;ll],
            Point2Point2RangeData(
              range=Normal(rho, 3.0)
          ))
      push!(resIds, addFactor(CliCon..., f));
    end
  end
  # wait to make sure all the new additions are ready
  waitForCompletion(client, resultIds; expectedStatuses=["Complete"])
  nothing
end

After running this function in Julia, a new member definition `vehicle_drives_to!` can be used line any other function. Julia will handle the just-in-time compiling for the type specific function required and cach the static code for repeat executions.

Now the actual driving event can be added to the factor graph:

In [ ]:
#drive to location :x1, then :x2
vehicle_drives(nvapl, "x0", "x1", GTp, GTl)
vehicle_drives(nvapl, "x1", "x2", GTp, GTl)

# see the graph
GraphVizApp(context)

**Note**, the distance traveled could be any combination of accrued direction and speeds, however, a straight line Gaussian error model is used to keep the visual presentation of this example as simple as possible.

Lets solve the whole factor graph again.

In [ ]:
push!(resultIds,
  solveSession(client, context)
);

# Give it a few seconds JIT compiling during first run.
println("running solve...")

### Multi-Modal (i.e. Multi-Hypothesis) Solution

Now lets look at the robot position marginal beliefs.  We'll use a slightly lower level plotting function, `plotBelief`, to show the posterior beliefs of the tree robot locations:

In [ ]:
# pl = plotBelief(fg, [:x0; :x1; :x2], levels=5, c=["red","green","blue"])
MapVizApp(context)

Notice how the robot trajectory has 2+ hypotheses,
- a) starting from `(0,0)` and traversing left to right, and 
- b) starting from `(50,-5)` and traversing left down.

Both are valid solutions!

### Resolving Estimates for New Beacons `l3` and `l4`

So far, we have added measurements to the **initially unknown beacons**:
- ranges to `l3` were measured from all three robot positions `[x0,x1,x2]`, and
- ranges to `l4` were only mearured from two robot positions `[x1, x2]`

Can you guess what the posterior belief estimate on new beacons `l3` or `l4` are given the available information up this point?  Let's plot and see...

In [ ]:
# pl = plotBelief(fg, [:l3;:l4], levels=5, c=["pink"; "orange"])
MapVizApp(context)

The two "free" beacons/landmarks `l3,l4` still have several modes each, implying insufficient data to constrain either to a conventional unimodal belief.  These marginal posteriors are clearly non-Gaussian.

## Robot Moves Two More Positions

The robot drives further to collect more information, keeping in mind that so far that only the two known beacons `l1` and `l2` have unimodal posterior belief estimates!  All other variables in the system `[l3;l4; x0;x1;x2]` have multi-modal belief

In [ ]:
vehicle_drives(nvapl, "x2", "x3", GTp, GTl)
vehicle_drives(nvapl, "x3", "x4", GTp, GTl)

push!(resultIds,
  solveSession(client, context)
);

# Give it a few seconds JIT compiling during first run.
println("running solve...")

### Estimating the Robot's Latest Position at `x3`, `x4`

After the above factor graph solution, the latest robot position belief estimate is 

In [ ]:
# pl = plotBelief(fg, [:x0;:x1;:x2;:x3;:x4], levels=5)
MapVizApp(context)

Notice how, even though several new range measurements were made, the posterior on `x3` has an increasing number of modes!  The number of modes may vary from solution to solution, but at least two dominant modes should be visible.

Modes are gained or lost based on a combination of the problem setup and nonparametric variations within each individual solve.  More dominant modes are more consistent, while 'weak' modes may come or go from one solve to the next.  If more compute resources are used, then more and more 'weaker' modes will be recovered.

> Several solver parameters can be modified to control the compute load vs. multimodal tracking efficacy.  Join the [Caesar.jl Slack](https://join.slack.com/t/caesarjl/shared_invite/zt-ucs06bwg-y2tEbddwX1vR18MASnOLsw) conversations, or connect with [NavAbility.io](https://www.navability.io/) to learn more.

The first robot position `[x0,x1]` belief estimates didn't change much with the addition new information!  There are still active hypotheses in the trajectory estimates going from `x0` to `x1`.  Perhaps the reason for that is because the new landmarks `[l3; l4]` are yet to be constrained to a low number of modes, lets see...

### Progress on Locating New Beacons `l3` and `l4`

We expect the uncertainty on position estimates for the initially unknown beacons `[l3;l4]` to decrease as new measurements are added to the overall problem.  Let's look again at the new posteriors on `l3` and `l4`:

In [ ]:
# pl = plotBelief(fg, [:l3;:l4], levels=5, c=["pink"; "orange"])
MapVizApp(context)

There are still multiple modes on both `l3` and `l4`!  We still have way too little information to resolve a unimodal estimate on either the robot positions or the new beacon locations!  The entire system remains underdetermined, i.e. singular!

### Moving to Positions `x5` and `x6`

The robot moves further through positions `x5` and `x6`, and let's solve again and look at the results:

In [ ]:
vehicle_drives(nvapl, "x4", "x5", GTp, GTl)
vehicle_drives(nvapl, "x5", "x6", GTp, GTl)

push!(resultIds,
  solveSession(client, context)
);

# Give it a few seconds JIT compiling during first run.
println("running solve...")

In [ ]:
# pl = plotBelief(fg, [Symbol("x$i") for i=0:6], levels=3)
MapVizApp(context)

#### Reviewing Landmark Location Estimates Again

In [ ]:
# pl = plotBelief(fg, [:l1;:l2;:l3;:l4], levels=5, c=["cyan";"black";"pink"; "orange"])
MapVizApp(context)

## Moving to Positions `x7` and `x8`

In [ ]:
vehicle_drives(nvapl, "x6", "x7", GTp, GTl)
vehicle_drives(nvapl, "x7", "x8", GTp, GTl)

In [ ]:

push!(resultIds,
  solveSession(client, context)
);

# Give it a few seconds JIT compiling during first run.
println("running solve...")

Using the list function, we can recover all the variables of interest, and then use for plotting:

In [ ]:
# @show lbls = sortDFG(ls(fg, r"x\d")); pl = plotBelief(fg, lbls, levels=5)
MapVizApp(context)

While the nonparametric solution has stochastic behaviou, the main modes of the position estimates are successfully tracked.  Smaller modes may also show up in some of the marginal estimates.  The correct solution does have multiple modes.  Part of our ongoing open core solver improvements is to improve mode tracking quality with less computation.  We stress that this tutorial is designed to showcase the variability from more modes, and that many practical applications can already benefit from this existing solver robustness to underdetermined situations.

Concurrently, the landmark estimates are also exhibit ongoing multimodality:

In [ ]:
# pl = plotBelief(fg, [:l1;:l2;:l3;:l4], levels=4, c=["cyan";"black";"pink"; "orange"])
MapVizApp(context)

There is still multi-modality on landmark position estimates!  This indicates the uncertain nature of the problem which is driven by weak observabilty.  To round out the tutorial, let's see what the factor graph looks like at this point:

In [ ]:
GraphVizApp(context)

## Conclusion

Theoretically, should there be one or more overall trajectory hypotheses in the final result?

## Next Steps

- This Tutorial showed [one of four](https://juliarobotics.org/Caesar.jl/latest/concepts/why_nongaussian/) identified mechanisms how non-Gaussian behavior can enter a localization and mapping system.  See the other tutorials and material for similar discussions on other mechanism by which multi-modal posteriors manifest.
- For a longer version of this example, see [open-source solver documentation here](https://juliarobotics.org/Caesar.jl/latest/examples/basic_slamedonut/).

Visit [www.NavAbility.io](https://www.NavAbility.io) for more about how to use these and other advanced navigation features in your application.